
# Phising web sites
goal is to construct a small yet useful decision tree that predicts whether a website is a phishing site or not.


# Import data
First import librarys and load data from csv and get column names to list

In [2]:
import pandas as pd
import numpy as np
import sklearn
import csv
from sklearn import tree
import graphviz
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
from IPython.display import IFrame

#read data
df = pd.read_csv('phishing.csv', sep=';')

#drop rows with missing data
df.dropna(axis=0, how='any', inplace=True)

# save column headings into a list
colnames = df.columns.get_values()

#df['Result'] = df['Result'].astype(pd.api.types.CategoricalDtype(ordered=False))
#df.dtypes()
df.head()

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWindow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1


# Extract
Choosing all columns to variable X and extract response variable 'Result' to Y
Then choose depht of classifier, which means how big the decision tree is

In [3]:
#Extract explanatory variables into a data frame
X = df.loc[:, 'having_IP_Address' : 'Statistical_report' ]

#Extract response variable
Y = df.loc[:, 'Result']

In [4]:
#Decision tree classification
classifier = tree.DecisionTreeClassifier(max_depth=2)
classifier.fit(X,Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

# Visualize
Visualizing tree by choosing all column names to tree and class_name tells how it would separate tree
Then render the image and choose how big it would be

In [5]:
# visualize
dot_data = tree.export_graphviz(classifier, out_file=None, feature_names=colnames[:30], class_names=['no','yes'])
graph = graphviz.Source(dot_data)
graph.render("Phising web sites 9")

IFrame("Phising web sites 9.pdf", width=900, height=800)

In [6]:
# Predict
Y_pred = classifier.predict(X)

#Output confusion matrix
cm = confusion_matrix(Y, Y_pred)
print("Confusion matrix: \n",cm)

accuracy = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0])
print("Accuracy from training set = %.3f" % (accuracy))

print(classification_report(Y, Y_pred, target_names=['No', 'Yes']))

Confusion matrix: 
 [[4425  473]
 [ 563 5594]]
Accuracy from training set = 0.906
              precision    recall  f1-score   support

          No       0.89      0.90      0.90      4898
         Yes       0.92      0.91      0.92      6157

   micro avg       0.91      0.91      0.91     11055
   macro avg       0.90      0.91      0.91     11055
weighted avg       0.91      0.91      0.91     11055



In [7]:
#cross-validate
#number of folds
k = 5
scores = cross_val_score(estimator=classifier,
                        X=X,
                        y=Y,
                        scoring="accuracy",
                        cv=k)
print("Accuracies from %d individual folds:" % k)
print(scores)
print("Accuracy calcuted using %d-fold cross validation = %.3f" % (k, scores.mean()))

Accuracies from 5 individual folds:
[0.89692586 0.90461121 0.91316147 0.90135747 0.91538462]
Accuracy calcuted using 5-fold cross validation = 0.906


In [55]:
# Experiment with min_samples_leaf to find the best model.

for leaf in range(201,-1,-20):
    print("min_samples_leaf: %d" % leaf)
    classifier = tree.DecisionTreeClassifier(min_samples_leaf=leaf)
    classifier.fit(X,Y)
    Y_pred = classifier.predict(X)
    cm = confusion_matrix(Y, Y_pred)
    accuracy = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0])
    print("  Accuracy calculated from the training set = %.3f" % (accuracy))
    
    scores = cross_val_score(estimator=classifier,
                        X=X,
                        y=Y,
                        scoring="accuracy",
                        cv=k)
    print("  Accuracy calculated using %d-fold cross validation = %.3f" % (k, scores.mean()))

min_samples_leaf: 201
  Accuracy calculated from the training set = 0.923
  Accuracy calculated using 5-fold cross validation = 0.908
min_samples_leaf: 181
  Accuracy calculated from the training set = 0.922
  Accuracy calculated using 5-fold cross validation = 0.914
min_samples_leaf: 161
  Accuracy calculated from the training set = 0.925
  Accuracy calculated using 5-fold cross validation = 0.912
min_samples_leaf: 141
  Accuracy calculated from the training set = 0.925
  Accuracy calculated using 5-fold cross validation = 0.915
min_samples_leaf: 121
  Accuracy calculated from the training set = 0.926
  Accuracy calculated using 5-fold cross validation = 0.919
min_samples_leaf: 101
  Accuracy calculated from the training set = 0.929
  Accuracy calculated using 5-fold cross validation = 0.916
min_samples_leaf: 81
  Accuracy calculated from the training set = 0.930
  Accuracy calculated using 5-fold cross validation = 0.918
min_samples_leaf: 61
  Accuracy calculated from the training se

# Instructions and results

First our decision tree shows that is web site have SSL sertificate and able to make SSL connection. 
next thing to review is URL of anchor which checks if website and anchor text have same domain name. 
gini index shows how tighly the observation in node fall into the same class if its zero all cases in the node fall into single target. So if SSL final state is less than 0.5 it has no SSL connection next step is if URL of anchor is less than 0.5 if thats also true then gini index is 0.0 which means that web site if phishing.